<a href="https://colab.research.google.com/github/catfoodlover/DATA604/blob/main/DATA604_Simulation_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on this practice exercise, other factors could be considerd to build a stronger simulations.  An imbalance in resource allocation could change the properties of the model.  Imagine you have too many cashiers but not enough servers or ushers.  If you are getting people into the theater faster than the system can accomodate it may slow down other parts of the system.  If there are too many people waiting to get their ticket taken it may actually increase the amount of time that it takes to process a ticket.  Or if people aren't getting there tickets taken fast enough they may skip getting snacks at the snack bar.  The relationship between the parts could be modeled more complexly.

Professionally I build models that show surgeons the perioperative and postoperate outcomes for a particular surgical procedure.  We know that outcomes can't be viewed in isolation.  It's been show accross services that surgeons who most aggressively discharge their patients postoperatively have the highest 30 readmission rates.  It should be noted that there are many good reasons to get patients home as quickly as possible (low risk of depression, infections, falled, delirium).  I could simulate the relationship between lenght of stay and readmission to see if there is an optimal lenght of stay that doesn't drastically increase the readmission rates.

In [21]:
# Set up the environment
import simpy
import random
import statistics

In [22]:
wait_times = []

In [23]:
class Theater(object):
    def __init__(self, env, num_cashiers):
        self.env = env
        self.cashier = simpy.Resource(env, num_cashiers)

    def purchase_ticket(self, moviegoer):
        yield self.env.timeout(random.randint(1, 3))


In [24]:
def go_to_movies(env, moviegoer, theater):
    # Moviegoer arrives at the theater
    arrival_time = env.now

    with theater.cashier.request() as request:
        yield request
        yield env.process(theater.purchase_ticket(moviegoer))

    with theater.usher.request() as request:
        yield request
        yield env.process(theater.check_ticket(moviegoer))

    if random.choice([True, False]):
        with theater.server.request() as request:
            yield request
            yield env.process(theater.sell_food(moviegoer))

    # Moviegoer heads into the theater
    wait_times.append(env.now - arrival_time)





In [25]:
def run_theater(env, num_cashiers, num_servers, num_ushers):
    theater = Theater(env, num_cashiers, num_servers, num_ushers)

    for moviegoer in range(3):
        env.process(go_to_movies(env, moviegoer, theater))

    while True:
        yield env.timeout(0.20)  # Wait a bit before generating a new person

        moviegoer += 1
        env.process(go_to_movies(env, moviegoer, theater))

In [26]:
def get_average_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)


In [27]:
def calculate_wait_time(arrival_times, departure_times):
    average_wait = statistics.mean(wait_times)
    # Pretty print the results
    minutes, frac_minutes = divmod(average_wait, 1)
    seconds = frac_minutes * 60
    return round(minutes), round(seconds)


In [28]:
def get_user_input():
    num_cashiers = input("Input # of cashiers working: ")
    num_servers = input("Input # of servers working: ")
    num_ushers = input("Input # of ushers working: ")
    params = [num_cashiers, num_servers, num_ushers]
    if all(str(i).isdigit() for i in params):  # Check input is valid
        params = [int(x) for x in params]
    else:
        print(
            "Could not parse input. The simulation will use default values:",
            "\n1 cashier, 1 server, 1 usher.",
        )
        params = [1, 1, 1]
    return params


In [29]:
def main():
  # Setup
  random.seed(42)
  num_cashiers, num_servers, num_ushers = get_user_input()

  # Run the simulation
  env = simpy.Environment()
  env.process(run_theater(env, num_cashiers, num_servers, num_ushers))
  env.run(until=90)

  # View the results
  mins, secs = get_average_wait_time(wait_times)
  print(
      "Running simulation...",
      f"\nThe average wait time is {mins} minutes and {secs} seconds.",
  )


In [30]:
if __name__ == '__main__':
    main()


Input # of cashiers working: 1
Input # of servers working: 1
Input # of ushers working: 1


TypeError: ignored